In [8]:
import pandas as pd 
import json
import numpy as np
from sklearn.metrics import precision_recall_fscore_support



In [5]:
from transformers import AutoTokenizer, pipeline
from custom_dataset import LedgerDataset

In [6]:
file_name = 'testing.xlsx'
save_json_name ='test.json'
model_name = "output_base_3/checkpoint-10000/"


In [4]:
df = pd.read_excel(file_name, sheet_name=None, index_col=None)
df = pd.concat(df.values(),ignore_index=True)
target_column = "target"
source_column = "source"

df = df.rename(columns={"Intelas Ledger Name": target_column, "Source Ledger Name": source_column})
df_to_save = df.apply(lambda x: {'id':str(x.name), 'translation': {'source' : x[source_column], 'target' : x[target_column]} }, axis=1)
with open(save_json_name, 'w') as fout:
    json.dump(list(df_to_save.to_dict().values()) , fout)

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
translator = pipeline("translation_source_to_target", model=model_name,max_length=30)
test_dataset = LedgerDataset(save_json_name,tokenizer)


In [10]:
prefix = "translate source to target: "
res = []
for item in test_dataset : 
    text = prefix + item["translation"]["source"]
    target_text = item["translation"]["target"]
    pred = translator(text)[0]['translation_text']
    res.append({
        "text":text, 
        "target" : target_text,
        "pred" : pred, 
    })

In [11]:
targets =[x["target"] for x in res]
predictions = [x["pred"] for x in res]

In [12]:
sum =0 
failed= 0 
for i in range(len(targets)): 
    if targets[i] == predictions[i]:
        sum+=1
    else : 
        failed +=1 
        print(f"target : {targets[i]}\n prediction : {predictions[i]}")
print("accuracy : ")
print(sum/len(targets))
print(failed/len(targets))

target : Restricted Cash - General
 prediction : Restricted Cash - Rehab Escrow
target : Intercompany Liability
 prediction : Intercompany Asset
target : Transfer / Sublet  Fee
 prediction : Transfer / Sublet Fee
target : Plumbing - Repairs
 prediction : Fixed Assets - General
target : Common Area - Exterior
 prediction : Fixed Assets - General
target : Common Area - General
 prediction : Fixed Assets - General
accuracy : 
0.9707317073170731
0.02926829268292683


In [13]:
precision_recall_fscore_support(np.array(targets), np.array(predictions), average='weighted')

/home/ali-mhd/task-ml-intelas/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ali-mhd/task-ml-intelas/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.963821138211382, 0.9707317073170731, 0.9641347270615563, None)